In [ ]:
##构造libffm所需要的dataformat
import pandas  as pd
from tqdm import tqdm
##怎么划分训练集和测试集
##训练集的要求 1 尽量包含完所有搜索词  先随便来试一下哦，不要管那么多
from sklearn.cross_validation import train_test_split
train_data_old = pd.read_csv('E:/competionfile/oppo/data/data_split/train_data_old.csv',encoding='utf8').astype(str)
test_data_old  = pd.read_csv('E:/competionfile/oppo/data/data_split/test_data_old.csv',encoding='utf8')
train_data_old.columns = ['prefix','query_prediction','title','tag','label']
test_data_old.columns = ['prefix','query_prediction','title','tag','label']
train_data_old[ 'label' ] = train_data_old['label'].astype(int)
train_data_old['realtitle'] = train_data_old['title']+train_data_old['tag']
test_data_old['realtitle'] = test_data_old['title']+test_data_old['tag']

train_data_old = train_data_old.reset_index()
train_data_old = train_data_old.drop(['index'],axis=1)
test_data_old = test_data_old.reset_index()
test_data_old = test_data_old.drop(['index'],axis=1)


train_data_new = pd.read_csv('E:/competionfile/oppo/data/data_split/train_data_new.csv',encoding='utf8').astype(str)
test_data_new  = pd.read_csv('E:/competionfile/oppo/data/data_split/test_data_new.csv',encoding='utf8').astype(str)

train_data_new[ 'label' ] = train_data_new['label'].astype(int)
train_data_new['realtitle'] = train_data_new['title']+train_data_new['tag']
test_data_new['realtitle'] = test_data_new['title']+test_data_new['tag']

train_data_new = train_data_new.reset_index()
train_data_new = train_data_new.drop(['index'],axis=1)
test_data_new = test_data_new.reset_index()
test_data_new = test_data_new.drop(['index'],axis=1)
tt = pd.read_csv('E:/competionfile/oppo/data/data_split/train_data_new.csv',encoding='utf8').astype(str)
train_data_new_label = tt.astype(str)['label'].astype(int)




In [ ]:
TestAllSim_Stopesult = pd.read_csv('E:/competionfile/oppo/data/doc2vec/TestAllSim_Stopesult.csv')
TrainAllSim_Stopesult = pd.read_csv('E:/competionfile/oppo/data/doc2vec/TrainAllSim_Stopesult.csv')
VailAllSim_Stopesult = pd.read_csv('E:/competionfile/oppo/data/doc2vec/VailAllSim_Stopesult.csv')

In [ ]:
TrainAllSim_Stopesult = TrainAllSim_Stopesult.append( TestAllSim_Stopesult , ignore_index=False )
TrainAllSim_Stopesult = TrainAllSim_Stopesult.append( VailAllSim_Stopesult , ignore_index=False )
from tqdm import tqdm
pre_Tsim_map = {}
query_Tsim_map = {}
for  item in tqdm(TrainAllSim_Stopesult.itertuples()):
    pre_Tsim_map[(item.prifix,item.title)] = item.pri_Tsim
    query_Tsim_map[(item.query,item.title)] = item.quer_Tsim
print(train_data_new.shape[0],train_data_old.shape[0])

In [ ]:
def getxx( item ):
#     try:
#         return pre_Tsim_map[(item.prefix,item.title)]
#     except:
#         print(item)
#     if item.prefix == 'nan':
#         return 0
#     else:
        return pre_Tsim_map[(item.prefix,item.title)]

train_data_new['pre_title_doc2vec_sim'] = train_data_new.apply( getxx  ,axis=1)
test_data_new['pre_title_doc2vec_sim'] = test_data_new.apply( lambda x: pre_Tsim_map[(x.prefix,x.title)],axis=1 )

In [ ]:
import json
def get_query_prediction_no1(predictStr):

    obj = json.loads(predictStr)
    if len(obj) == 0:
        return None
    curlist = []
    for  i in obj:
        curlist.append( ( i , float(obj[i]) ) )
    curlist.sort( key = lambda x : x[1] , reverse = True )
    return curlist[0][0]
train_data_new['query_prediction_no1'] = train_data_new.apply( lambda x: get_query_prediction_no1( x.query_prediction ) ,axis=1)
test_data_new['query_prediction_no1'] = test_data_new.apply(  lambda x: get_query_prediction_no1( x.query_prediction ) ,axis=1 )

train_data_new['query_title_doc2vec_sim'] = train_data_new.apply( lambda x: query_Tsim_map.get((x.query_prediction_no1,x.title),0) ,axis=1)
test_data_new['query_title_doc2vec_sim'] = test_data_new.apply( lambda x: query_Tsim_map.get((x.query_prediction_no1,x.title),0) ,axis=1)

print(train_data_new.shape[0],train_data_old.shape[0])

In [ ]:
## 做一下几个特征，搜索词长度的点击率，搜索词长度，文章长度，搜索词减文章长度，title点击率，titletag点击率，编辑距离，语义相似度（预测词第一名和， query和title的相似度, tag点击率 ），搜索词是否在预测词中出现过 ，
import difflib
import Levenshtein

def get_equal_rate_1(str1,str2):
    return difflib.SequenceMatcher(None, str1, str2).quick_ratio()

def latt( s1 , s2 ):
    return Levenshtein.distance(s1,s2)

train_data_old['title_len'] = train_data_old['title'].map( lambda x : len(x) )
title_len_ctr_dict = train_data_old.groupby('title_len')['label'].mean()
title_ctr_dict = train_data_old.groupby('title')['label'].mean()
title_tag_ctr_dict = train_data_old.groupby(['title','tag'])['label'].mean()
tag_ctr_dict = train_data_old.groupby('tag')['label'].mean()


title_len_ctr_dict_idx = title_len_ctr_dict.index

train_data_new['prefix_len'] = train_data_new['prefix'].map( lambda x : len(x) )
train_data_new['title_len'] = train_data_new['title'].map( lambda x : len(x) )
train_data_new['title_len_ctr'] = train_data_new['title_len'].map( lambda x : 0.37 if x not in title_len_ctr_dict_idx else title_len_ctr_dict[x]  )
train_data_new['title_len_sub_prefix_len'] = train_data_new['title_len'] - train_data_new['prefix_len']
train_data_new['title_ctr'] = train_data_new['title'].map( lambda x : 0.37 if x not in title_ctr_dict.index else title_ctr_dict[x] )
# train_data_new['title_tag_ctr'] = train_data_new.apply( lambda x : 0.37 if (x['title'],x['tag']) not in title_tag_ctr_dict.index else title_tag_ctr_dict[(x['title'],['tag'])]  , axis=1 )
train_data_new['prefix_title_difsim'] = train_data_new.apply( lambda x : get_equal_rate_1( x['prefix'] , x['title'] )  , axis=1 )
train_data_new['prefix_title_latsim'] = train_data_new.apply( lambda x : latt( x['prefix'] , x['title'] )  , axis=1 )
train_data_new['title_ocrin_prediction'] = train_data_new.apply( lambda x : 1 if x['title'] in x['query_prediction'] else 0    , axis=1 )

train_data_new['tag_ctr'] = train_data_new['tag'].map( lambda x : tag_ctr_dict[x]  )

print(train_data_new.shape[0],train_data_old.shape[0])

In [ ]:
test_data_new['prefix_len'] = test_data_new['prefix'].map( lambda x : len(x) )
test_data_new['title_len'] = test_data_new['title'].map( lambda x : len(x) )
test_data_new['title_len_ctr'] = test_data_new['title_len'].map( lambda x : 0.37 if x not in title_len_ctr_dict_idx else title_len_ctr_dict[x]  )
test_data_new['title_len_sub_prefix_len'] = test_data_new['title_len'] - train_data_new['prefix_len']
test_data_new['title_ctr'] = test_data_new['title'].map( lambda x : 0.37 if x not in title_ctr_dict.index else title_ctr_dict[x] )
# train_data_new['title_tag_ctr'] = train_data_new.apply( lambda x : 0.37 if (x['title'],x['tag']) not in title_tag_ctr_dict.index else title_tag_ctr_dict[(x['title'],['tag'])]  , axis=1 )
test_data_new['prefix_title_difsim'] = test_data_new.apply( lambda x : get_equal_rate_1( x['prefix'] , x['title'] )  , axis=1 )
test_data_new['prefix_title_latsim'] = test_data_new.apply( lambda x : latt( x['prefix'] , x['title'] )  , axis=1 )
test_data_new['title_ocrin_prediction'] = test_data_new.apply( lambda x : 1 if x['title'] in x['query_prediction'] else 0    , axis=1 )

test_data_new['tag_ctr'] = test_data_new['tag'].map( lambda x : tag_ctr_dict[x]  )



In [ ]:
train_y_data = train_data_new['label']
train_x_data = train_data_new.drop(['prefix','query_prediction','title','tag','label','query_prediction_no1','realtitle'] , axis = 1 )
test_x_data = test_data_new.drop(['prefix','query_prediction','title','tag','label','query_prediction_no1','realtitle'] , axis = 1 )
test_y_data = test_data_new['label'].astype(int)

In [ ]:
base_dir = 'E:/competionfile/oppo/data/data_split/'
train_x_data.to_csv('base_dir'+'train_data_new_x_data.csv',index=False)
test_x_data.to_csv('base_dir'+'test_data_new_x_data.csv',index=False)
test_y_data.to_csv('base_dir'+'test_data_new_y_data.csv',index=False)
train_y_data.to_csv('base_dir'+'train_data_new_y_data.csv',index=False)
